In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


C:\Users\jfraz\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\jfraz\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
appcounts = application_df['APPLICATION_TYPE'].value_counts()
appcounts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(appcounts[appcounts<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
appbinning = application_df['CLASSIFICATION'].value_counts()
appbinning

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x :x>1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C2700', 'C7000', 'C7200','C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C1238','C5000', 'C7120', 'C1800', 'C4100', 'C1400','C1270',
'C1300','C1280','C1230','C2300','C1240','C8000','C1500','C1250','C8200','C1278','C1235','C1237','C7210','C1720','C2400','C1257','C1600','C1260','C2710','C0','C1267','C3200','C1234','C1246','C1256','C2190','C4200','C1248','C5200','C1370','C2600','C6100','C1820','C1900','C1236','C1245','C2570','C1580','C3700','C2500','C1570','C1283','C2380','C1732','C1728','C2170','C4120','C8210','C4500','C2561', 
'C2150']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df)

In [ ]:
# Split our preprocessed data into our features and target arrays
y = app_dummies['IS_SUCCESSFUL'].values
X = app_dummies.drop(['IS_SUCCESSFUL'],axis=1)

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_features = len(X_train_scaled[0])
hidden_nodes_1 = 80
hidden_nodes_2 = 30


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, input_dim=num_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

In [ ]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#checkpoint path and filename 
os.makedirs('checkpoints/',exist_ok= True)
checkpoint_path = 'checkpoints/weights.{epoch:05d}.hdf5'

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
cpcallback= ModelCheckpoint(filepath =checkpoint_path,verbose=5,save_weights_only=True,period = 5)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cpcallback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Create a DataFrame containing training history
history_df = pd.DataFrame(fit_model.history, index=range(1,len(fit_model.history["loss"])+1))

# Plot the accuracy
history_df.plot(y="accuracy")

In [ ]:
# Export our model to HDF5 file
nn.save('./Resources/AlphabetSoupCharity.h5')